# Parte Comum

In [1]:
import pandas as pd
import glob as gb
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from math import pi
from windrose import WindroseAxes
import os
from IPython.display import clear_output
import plotly.express as px


caminho = "C:\\Users\\romulo24024\\Downloads\\Atividades\\Respostas\\PCD\\Projeto Final 1º Semestre\\"

dicionario = {
    '01' : {
        'nome': 'Janeiro',
    },
    '02' : {
        'nome': 'Fevereiro',
    },
    '03': {
        'nome': 'Março',
    },
    '04': {
        'nome': 'Abril',
    },
    '05': {
        'nome': 'Maio',
    },
    '06': {
        'nome': 'Junho',
    },
    '07': {
        'nome': 'Julho',
    },
    '08': {
        'nome': 'Agosto',
    },
    '09': {
        'nome': 'Setembro',
    },
    '10': {
        'nome': 'Outubro',
    },
    '11': {
        'nome': 'Novembro',
    },
    '12': {
        'nome': 'Dezembro',
    },
}
nome_meses =  []
meses = []

for i in dicionario:
    nome_meses.append(dicionario[i]['nome'])
    
for i in range (1,13):
    if i < 10:
        i = f"0{i}"
    meses.append(str(i))

# Funções Fundamentais

In [2]:
def escolhe_cidade():
    cidade_escolhida = str(input("Escolha uma estação do INMET (use _ como espaço): "))
    global arquivo_escolhido
    global dados
    global cidade

    arquivo_escolhido = None  
    

    for i in range(len(cidades)):
        
        if cidade_escolhida in cidades[i]:

            arquivo_escolhido = arquivos[i]

    if arquivo_escolhido:

        clear_output(wait=False)
    
    
    with open(f"{caminho}\\{ano}\\{arquivo_escolhido}",
        encoding='utf-8', errors='ignore') as tabela:
        dados = pd.read_csv(tabela, decimal = ',', sep = ";", header = 8,na_values= [-9999])
        
    with open(f"{caminho}\\{ano}\\{arquivo_escolhido}",
      encoding='utf-8', errors='ignore') as tabela:
        cabecalho = pd.read_csv(tabela, nrows = 8, header = None,sep = ';', na_values= [-9999]).transpose()
        cidade = cabecalho[2].iloc[1].replace(" ","_").replace("(","").replace(")","").replace("-","_")
        cidade = cidade.replace("__","_")

def escolhe_ano():
    global ano
    global cidades
    global arquivos
    global cidade
    global cabecalho
    global dados
    
    cidades = []
    ano = int(input("Insira o ano:\n"))
    clear_output(wait=False)
    
    os.chdir(f"{caminho}\\{ano}\\")
    arquivos = gb.glob('INMET_*.csv')
    
    for arquivo in arquivos:
        with open(f'{arquivo}', encoding='utf-8', errors='ignore') as tabela:
            cabecalho = pd.read_csv(tabela, nrows = 8, header = None,sep = ';').transpose()
            cidade = cabecalho[2].iloc[1].replace(" ","_").replace("(","").replace(")","").replace("-","_")
            for letra in cidade:
                cidade = cidade.replace("__","_")
            cidades.append(cidade)
        with open(f'{arquivo}', encoding='utf-8', errors='ignore') as tabela:
            dados = pd.read_csv(tabela, header = 8,sep = ';', na_values= [-9999])


def escolhe_tempo():
    dia_0 = int(input("Insira o dia de início:\n"))
    if dia_0 < 10:
        dia_0 = str(f'0{dia_0}')
        
    mes_0 = int(input("Insira o mês de início:\n"))
    if mes_0 < 10:
        mes_0 = str(f'0{mes_0}')
        
    dia_1 = int(input("Insira o dia de término:\n"))
    if dia_1 < 10:
        dia_1 = str(f'0{dia_1}')
        
    mes_1 = int(input("Insira o mês de término:\n"))
    if mes_1 < 10:
        mes_1 = str(f'0{mes_1}')
    
        
    
    
    global tempo_final
    global tempo_inicial
    
    tempo_inicial = f'{ano}/{mes_0}/{dia_0}'
    tempo_final = f'{ano}/{mes_1}/{dia_1}'
    
    clear_output(wait=False)

def segmenta_mes():
    for mes in meses:
        globals()[f'dados_{mes}'] = pd.DataFrame() # Cria uma tabela vazia para cada mês
        for linha in range (0, len(dados['Data'])): # Vou analisar cada elemento da coluna data
            if dados['Data'][linha][5:7] == mes: # Se do 6º ao 8º caractere (correspondentes ao mês) forem iguais à string mês
                globals()[f'dados_{mes}'] = pd.concat([globals()[f'dados_{mes}'], dados.iloc[[linha]]]) # Aqui adiciona a linha à tabela do mês


    
def segmenta_variavel():
    global dados_referencia
    global linha_inicial
    global linha_final
    
    with open(f"{caminho}\\{ano}\\{globals()[f'arquivo_escolhido']}",
              encoding='utf-8', errors='ignore') as tabela:
        globals()[f'dados'] = pd.read_csv(tabela, decimal = ',', sep = ";", header = 8)
        
    dados_referencia = pd.DataFrame()
    for linha in range (0, len(dados)):
        if dados['Data'][linha] == tempo_inicial:
            linha_inicial = linha
            break

    for linha in range (0, len(dados)):
        if dados['Data'][linha] == tempo_final:
            linha_final = linha

    for linha in range (linha_inicial, linha_final):
        dados_referencia = pd.concat([dados_referencia, dados.iloc[[linha]]])# Aqui adiciona a linha à tabela do mês
        
    global lista_dias
    lista_dias = []
    for linha in range(linha_inicial, linha_final):
        dia = dados_referencia['Data'][linha][8:]
        mes = dados_referencia['Data'][linha][5:7]

        if f'{dia}/{mes}' not in lista_dias:
            globals()[f'dados_{dia}_{mes}'] = pd.DataFrame()
            lista_dias.append(f'{dia}/{mes}')
        else:
            globals()[f'dados_{dia}_{mes}'] = pd.concat([globals()[f'dados_{dia}_{mes}'], dados_referencia.loc[[linha]]])
            
def comparativo():
    global n_cidades
    n_cidades = int(input("Selecione o número de cidades"))
    global lista_cidades
    lista_cidades = []

    
    for i in range (0, n_cidades):
        globals()[f'cidade_escolhida_{i}'] = str(input("Escolha uma estação do INMET (use _ como espaço): "))
        print(globals()[f'cidade_escolhida_{i}'])
        globals()[f'arquivo_escolhido{i}'] = None

        for j in range(len(cidades)):

            if globals()[f'cidade_escolhida_{i}'] in cidades[j]:                    
                globals()[f'arquivo_escolhido{i}'] = arquivos[j]

        if globals() [f'arquivo_escolhido{i}']:

            clear_output(wait=False)

        with open(f"{caminho}\\{ano}\\{globals()[f'arquivo_escolhido{i}']}",
            encoding='utf-8', errors='ignore') as tabela:
            globals()[f'dados_{i}'] = pd.read_csv(tabela, decimal = ',', sep = ";", header = 8, na_values= [-9999])
        with open(f"{caminho}\\{ano}\\{globals()[f'arquivo_escolhido{i}']}",
          encoding='utf-8', errors='ignore') as tabela:
            cabecalho = pd.read_csv(tabela, nrows = 8, header = None,sep = ';').transpose()
            globals()[f'cidade_{i}'] = cabecalho[2].iloc[1].replace(" ","_").replace("(","").replace(")","").replace("-","_")
            globals()[f'cidade_{i}'] = cidade.replace("__","_")
        
        lista_cidades.append(globals()[f'cidade_escolhida_{i}'])

# Funções de Plot

In [6]:
def climograma_mes():
    global acumulado_precipitacao_mensal
    acumulado_precipitacao_mensal = []
    media_temperatura = []
    

    for mes in meses:
        acumulado_precipitacao_mensal.append(globals()[f'dados_{mes}']['PRECIPITAO TOTAL, HORRIO (mm)'].sum())
        temperatura_minima = globals()[f'dados_{mes}']['TEMPERATURA MNIMA NA HORA ANT. (AUT) (C)']
        temperatura_maxima = globals()[f'dados_{mes}']['TEMPERATURA MXIMA NA HORA ANT. (AUT) (C)']
        media_temp = (temperatura_minima.mean() + temperatura_maxima.mean()) / 2
        media_temperatura.append(media_temp)
            
    fig = make_subplots(specs=[[{'secondary_y': True}]])

    fig.add_trace(go.Bar(x=nome_meses,
                         y=acumulado_precipitacao_mensal,
                         name='Precipitação média',
                         marker_color='#1D8DFC'),
                  secondary_y=False)

    fig.add_trace(go.Scatter(x=nome_meses,
                             y= media_temperatura,
                             name='Temperatura média',
                             mode='lines+markers',
                             line=dict(color='red')),
                  secondary_y=True)

    fig.update_xaxes(title_text='Mes')
    fig.update_yaxes(title_text='P (mm)', secondary_y=False)
    fig.update_yaxes(title_text='Temperatura (°C)', secondary_y=True)

    fig.update_layout(title=f'Climograma do ano {ano} em {cidade[0]}{cidade[1:].lower()}',
                      template='seaborn')

    # Exibindo o gráfico
    fig.show()
def climograma_variavel():
    acumulado_precipitacao = []
    media_temperatura = []

    for dia in lista_dias:
        dia = dia.replace("/","_")
        acumulado_precipitacao.append(globals()[f'dados_{dia}']['PRECIPITAO TOTAL, HORRIO (mm)'].sum())
        temperatura_minima = globals()[f'dados_{dia}']['TEMPERATURA MNIMA NA HORA ANT. (AUT) (C)']
        temperatura_maxima = globals()[f'dados_{dia}']['TEMPERATURA MXIMA NA HORA ANT. (AUT) (C)']
        media_temp = (temperatura_minima.mean() + temperatura_maxima.mean()) / 2
        media_temperatura.append(media_temp)
            
    fig = make_subplots(specs=[[{'secondary_y': True}]])

    fig.add_trace(go.Bar(x=lista_dias,
                         y=acumulado_precipitacao,
                         name='Precipitação média',
                         marker_color='#1D8DFC'),
                  secondary_y=False)

    fig.add_trace(go.Scatter(x=lista_dias,
                             y= media_temperatura,
                             name='Temperatura média',
                             mode='lines+markers',
                             line=dict(color='red')),
                  secondary_y=True)

    fig.update_xaxes(title_text='Mes')
    fig.update_yaxes(title_text='P (mm)', secondary_y=False)
    fig.update_yaxes(title_text='Temperatura (°C)', secondary_y=True)

    fig.update_layout(title=f'Climograma do ano {ano} em {cidade[0]}{cidade[1:].lower()}',
                      template='seaborn')

    # Exibindo o gráfico
    fig.show()
    
def rosa_mes():
    for mes in meses:
        globals()[f'vento_velocidade_{mes}'] = globals()[f'dados_{mes}']['VENTO, VELOCIDADE HORARIA (m/s)']
        globals()[f'vento_direcao_{mes}'] = globals()[f'dados_{mes}']['VENTO, DIREO HORARIA (gr) ( (gr))']
        nome_mes = dicionario[mes]['nome']
        ax = WindroseAxes.from_ax()
        ax.bar(globals()[f'vento_direcao_{mes}'], globals()[f'vento_velocidade_{mes}'], normed=True, opening=0.8, edgecolor='white')
        ax.set_legend()
        ax.set_title(f'Rosa dos ventos em {cidade[0]}{cidade[1:].lower()}, {nome_mes} {ano}', size=20, color = '#483d8b')

def rosa_variavel():
    ax = WindroseAxes.from_ax()
    bins = [0,2,4,6,8]
    ax.bar(dados_referencia['VENTO, DIREO HORARIA (gr) ( (gr))'],dados_referencia['VENTO, VELOCIDADE HORARIA (m/s)'], normed=True, opening=0.8, edgecolor='white',bins = bins)
    ax.set_legend()
    ax.set_title(f'Rosa dos ventos em {cidade[0]}{cidade[1:].lower()} entre {lista_dias[0]} e {lista_dias[-1]}', size=20, color = '#483d8b')

def rosa_comparativa():
    for i in range(0, n_cidades):
        ax = WindroseAxes.from_ax()
        ax.bar(globals()[f'dados_{i}']['VENTO, DIREO HORARIA (gr) ( (gr))'],globals()[f'dados_{i}']['VENTO, VELOCIDADE HORARIA (m/s)'] , normed=True, opening=0.8, edgecolor='white')
        ax.set_legend()
        ax.set_title(f'Rosa dos ventos do ano de {ano} em {lista_cidades[i][0]}{lista_cidades[i][1:].lower()}', size=20, color = '#483d8b')

def plota_radiacao():
    cidades = []
    for arquivo in arquivos:
        with open(f'{arquivo}', encoding='utf-8', errors='ignore') as tabela:
            cabecalho = pd.read_csv(tabela, nrows = 8, header = None,sep = ';').transpose()
        with open(f'{arquivo}', encoding='utf-8', errors='ignore') as tabela:
            dados = pd.read_csv(tabela, header = 8,sep = ';', na_values= [-9999])
        cidade = cabecalho[2].iloc[1].replace(" ","_").replace("(","").replace(")","").replace("-","_")
        cidade = cidade.replace("__","_")
        cidades.append(cidade)
        globals()[f'latitude_{cidade}'] = cabecalho[4][1]
        globals()[f'longitude_{cidade}'] = cabecalho[5][1]
        globals()[f'radiacao_{cidade}'] = dados['RADIACAO GLOBAL (Kj/m)']
        
    radiacao_cidades_dataframe = pd.DataFrame()
    radiacao_cidades_dataframe = [globals()[(f'radiacao_{i}')] for i in cidades] #lista com um dataframe da radiação de cada cidade
    radiacao_cidades = []

    for i in range(len(radiacao_cidades_dataframe)):
        numeric_values = pd.to_numeric(radiacao_cidades_dataframe[i], errors='coerce')
        media = np.nanmean(numeric_values)
        radiacao_cidades.append(media)

    latitude_cidades = [globals()[(f'latitude_{i}')] for i in cidades]
    longitude_cidades = [globals()[(f'longitude_{i}')] for i in cidades]
    latitude_cidades = [float(str(lat).replace(',', '.')) for lat in latitude_cidades]
    longitude_cidades = [float(str(lon).replace(',', '.')) for lon in longitude_cidades]

    # Criar um DataFrame com os dados
    df_cidades = pd.DataFrame({
        'cidade': cidades,
        "latitude": latitude_cidades,
        "longitude": longitude_cidades,
        'radiacao': radiacao_cidades
    })
    # URL do arquivo GeoJSON dos estados brasileiros
    geojson_url = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"

    # Criar o mapa dos estados sem coloração
    fig = px.choropleth(
        geojson=geojson_url,
        locations=['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO'],
        featureidkey="properties.sigla",
        color_discrete_sequence=["white"],
        hover_name=['Acre', 'Alagoas', 'Amazonas', 'Amapá', 'Bahia', 'Ceará', 'Distrito Federal', 'Espírito Santo', 'Goiás', 'Maranhão', 'Minas Gerais', 'Mato Grosso do Sul', 'Mato Grosso', 'Pará', 'Paraíba', 'Pernambuco', 'Piauí', 'Paraná', 'Rio de Janeiro', 'Rio Grande do Norte', 'Rondônia', 'Roraima', 'Rio Grande do Sul', 'Santa Catarina', 'Sergipe', 'São Paulo', 'Tocantins']
    )

    # Adicionar os pontos das cidades com os valores de radiação
    fig.add_trace(
        go.Scattergeo(
            lon=df_cidades['longitude'],
            lat=df_cidades['latitude'],
            hovertext=df_cidades.apply(lambda row: f"{row['cidade']}<br>Radiação: {row['radiacao']} W/m²", axis=1),
            mode='markers',
            marker=dict(
                size=8,
                color=df_cidades['radiacao'],
                colorscale=[
                    [0, 'yellow'],    # Amarelo
                    [1, 'darkred']    # Vermelho escuro
                ],
                colorbar=dict(title="Radiação")
            )
        )
    )

    # Customizar o layout do mapa
    fig.update_geos(
        fitbounds="locations",
        visible=False
    )
    fig.update_layout(
        title_text= f"Mapa do Brasil com Valores de Radiação nas Cidades no ano de {ano}",
        title_x=0.5
    )

    # Exibir o mapa
    fig.show()
 

# Inserção do usuário

In [7]:
 tipo = str(input("Qual tipo de gráfico?\n 1 - Climograma Anual \n 2 - Climograma de intervalo variável \n 3 - Rosa dos Ventos Anual\n 4 - Rosa dos Ventos de intervalo variável\n 5 - Rosa dos Ventos comparativa\n 6 - Radiação\n"))

if tipo == str(1):
    escolhe_ano()
    escolhe_cidade()
    segmenta_mes()
    climograma_mes()
    
if tipo == str(2):
    escolhe_ano()
    escolhe_cidade()
    escolhe_tempo()
    segmenta_variavel()
    climograma_variavel()

if tipo == str(3):
    escolhe_ano()
    escolhe_cidade()
    segmenta_mes()
    rosa_mes()
    
if tipo == str(4):
    escolhe_ano()
    escolhe_cidade()
    escolhe_tempo()
    segmenta_variavel()
    rosa_variavel()
    
if tipo == str(5):
    escolhe_ano()
    escolhe_tempo()
    comparativo()
    rosa_comparativa()
    
if tipo == str(6):
    escolhe_ano()
    plota_radiacao()

C:\Users\romulo24024\AppData\Local\Temp\ipykernel_25020\1779150634.py:119: RuntimeWarning:

Mean of empty slice

